In [1]:
import pymysql
import yfinance as yf
import import_ipynb

In [2]:
from settings import conn

importing Jupyter notebook from settings.ipynb


In [3]:
conn = conn

In [4]:
cursor = conn.cursor()

In [5]:
def create(sql):
    cursor.execute(sql)

In [6]:
def stock_delete(stock_code):
    sql = "DELETE FROM stock where code = '{0}'".format(stock_code)
    cursor.execute(sql)
    conn.commit()

# quarter, month, day5 table create

In [7]:
for i in ['quarter', 'month', 'day5']:
    sql = "CREATE TABLE IF NOT EXISTS {0}(stock_code varchar(9) not null,start varchar(50) not null, end varchar(50) not null, avg_volume bigint not null, avg_price bigint not null, avg_sd_volume double not null, avg_sd_price double not null, primary key (stock_code), foreign key (stock_code) references stock(code) on update cascade on delete cascade)".format(i)
    create(sql)
    conn.commit()

# record table create

In [8]:
sql = "CREATE TABLE IF NOT EXISTS record(stock_code varchar(9) not null, date varchar(50) not null, volume bigint not null, price bigint not null, primary key (stock_code, date), foreign key (stock_code) references stock(code) on update cascade on delete cascade)"
create(sql)

# table insert

In [9]:
sql = "SELECT * FROM stock"

In [10]:
cursor.execute(sql)
stocks = cursor.fetchall()
conn.commit()

In [11]:
sql = "INSERT INTO record (stock_code, date, volume, price) VALUES (%s, %s, %s, %s)"

In [ ]:
for j in stocks:
    try:
        d = yf.Ticker(j[0]+".KS")
        transaction = d.history("3mo")
        transaction = transaction.reset_index()
        transaction = transaction[['Date', 'Close', 'Volume']]
        transaction['Date'] = transaction['Date'].dt.strftime("%Y-%m-%d")
        transaction['Close'] = transaction['Close'].astype('int')
        for k in range(len(transaction)):
            cursor.execute(sql, (j[0], transaction['Date'][k], transaction['Volume'][k], transaction['Close'][k]))
    except:
        stock_delete(j[0]) # yfinance에서 지원하지 않는 종목은 삭제
conn.commit()

- 001000.KS: No data found, symbol may be delisted
- 001540.KS: No data found, symbol may be delisted
- 001810.KS: No data found, symbol may be delisted
- 001840.KS: No data found, symbol may be delisted
- 002230.KS: No data found, symbol may be delisted


KeyboardInterrupt: 

In [5]:
sql = "SELECT * FROM stock"

In [6]:
cursor.execute(sql)
stocks = cursor.fetchall()
conn.commit()

In [8]:
import numpy as np

In [28]:
for i in stocks:
    sql = "SELECT * FROM record WHERE stock_code = {0} ORDER BY date".format(i[0])
    cursor.execute(sql)
    record = cur.fetchall()

    record_day5 = record[-4:]
    start = record_day5[0][1]
    end = record_day5[-1][1]
    record_day5 = [(j[2], j[3]) for j in record_day5]
    avg_volume, avg_price = np.mean(record_day5, axis = 0)
    avg_sd_volume, avg_sd_price = np.std(record_day5, axis = 0)
    cursor.execute("INSERT INTO day5 (stock_code, start, end, avg_volume, avg_price, avg_sd_volume, avg_sd_price) VALUES (%s, %s, %s, %s, %s, %s, %s)", (i[0], start, end, avg_volume, avg_price, round(avg_sd_volume, 1), round(avg_sd_price, 1)))

    record_month = record[-29:]
    start = record_month[0][1]
    end = record_month[-1][1]
    record_month = [(j[2], j[3]) for j in record_month]
    avg_volume, avg_price = np.mean(record_month, axis = 0)
    avg_sd_volume, avg_sd_price = np.std(record_month, axis = 0)
    cursor.execute("INSERT INTO month (stock_code, start, end, avg_volume, avg_price, avg_sd_volume, avg_sd_price) VALUES (%s, %s, %s, %s, %s, %s, %s)", (i[0], start, end, avg_volume, avg_price, round(avg_sd_volume, 1), round(avg_sd_price, 1)))

    start = record[0][1]
    end = record[-1][1]
    record = [(j[2], j[3]) for j in record]
    avg_volume, avg_price = np.mean(record, axis = 0)
    avg_sd_volume, avg_sd_price = np.std(record, axis = 0)
    cursor.execute("INSERT INTO quarter (stock_code, start, end, avg_volume, avg_price, avg_sd_volume, avg_sd_price) VALUES (%s, %s, %s, %s, %s, %s, %s)", (i[0], start, end, avg_volume, avg_price, round(avg_sd_volume, 1), round(avg_sd_price, 1)))
    
    conn.commit()

In [44]:
conn.close()